# Greedy search

In [1]:
import sys
import queue as Q

## Setting up the map

*If you have already set up the map from the breadth-first search notebook you can copy the code and move on to the `performing the search` section.*

Before we can perform a search we need a map to search on!
The next few functions set up the map and help us perform some calculations as to if
a move is possible, who a nodes neighbours are and what the shortest path is after the goal has been found.

### The `in_bounds` function

This function takes a potential node and checks if it is bounds of the map.
If the node is in bounds return `true`, else `false`.

In [2]:
def in_bounds(node_id):
    (x, y) = node_id
    return 0 <= x < width and 0 <= y < height

### The `passable` function

This function takes a potential node and checks it against the list of walls.
If the node is a wall return `false`, else `true`.
This will prevent the evaluation of non accessible nodes

In [3]:
def passable(node_id):
    return node_id not in walls

### The `neighbours` function

This function take a node and returns its neighbours. As we are using Manhattan distance
diagonal movement is not allowed. The neighbours therefore before the nodes above, below, to the left and to the right of the original node. Each neighbour is checked to determine if it is in bounds and passable using the `in_bounds` and `passible` function.

In [4]:
def neighbors(node_id):
    (x, y) = node_id
    results = [(x + 1, y), (x, y - 1), (x - 1, y), (x, y + 1)]
    if (x + y) % 2 == 0: results.reverse()  # aesthetics
    results = filter(in_bounds, results)
    results = filter(passable, results)
    return results

### The `reconstruct_path` function

This function takes an array of parents e.g. Using the goal node locaiton as a key will return the value of its parent. This can be used to retrace the algorithms steps, from the goal node to the start, to get the shortest path the algorithm found.

As `came_from` is a mapping of parent -> child we need to start at the goal and work backwards. Therefore the `current` is set to the `goal` node and an empty `path` array is created. 

A `while` loop allows us to work backwards until we have reached the `start` node again.
The current node is added to the `path` array and we work backwards by then setting `current` to `came_from[current]`. This will set `current` to the child of `current`

In [5]:
def reconstruct_path(came_from):
    current = end
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

### The 'draw_grid' function

This function is a printing function to visualise what the algorithm is doing in the command line.
The `current` parameter is the current node the algorithm is evaluating; the `visited` parameter is an array of locations the algorithm has found.
The dimensions of the map are iterative over, the console outputs a `#` for a wall; `C` for the current location; `S` for the starting location; `E` for the end location; `V` for a node visited and `.` otherwise.

In [6]:
def draw_grid(current, visited):
    for y in range(height):
        for x in range(width):
            draw_node = (x, y)

            if draw_node in walls:
                sys.stdout.write(" # ")
            elif draw_node == current:
                sys.stdout.write(" C ")
            elif draw_node == start:
                sys.stdout.write(" S ")
            elif draw_node == end:
                sys.stdout.write(" E ")
            elif draw_node in visited:
                sys.stdout.write(" V ")
            else:
                sys.stdout.write(" . ")
        sys.stdout.write("\n")

## Performing the search

Now we have all the functions in place to create the map and perform some basic calculations to give us information about the map we are able to perform a search.

### The `calculate_manhattan_distance` function 

This function calculates the distance between two points. The Manhattan distance can only travel up, down, left and right, not diagonal. The value returned from this function will be out heuristic value.

Manhattan distance = $|x_1 - y_1| + |x_2 - y_2|$

For information on how to calculate distance using diagonals look up, Euclidean distance and Chebyshev distance and how they differ.

In [7]:
def calculate_manhattan_distance(n, goal):
    (n_x, n_y) = n
    (goal_x, goal_y) = goal
    return abs(n_x - goal_x) + abs(n_y - goal_y)

### The `greedy_search` function

This function is the main part of the search algorithm.
It maintains two lists, an `unexplored` list and a `found` list.

The `unexplored` list contains new nodes the algorithm comes across and has not seen or evaluated. At first, the only node in the list is the starting node. Entries into the list are sorted based on their heuristic value, $h(n)$. The head of the list is the node with the lowest $h(n)$ value as this is the node closest to the goal.

The `found` dictionary contains all the nodes that have found but they might or might not have been explored. The dictionary uses a node as a key. The key is a child and the value is its parent. E.g. using the starting node as a key will return the value `None` as the starting node has no parents, but using the goal node as a key will return its parent used to get there. This is important as we need this layout to be able to obtain the shortest path the algorithm can find. 

#### Implementation

The function starts off creating an empty `unexplored` list using the data structure, priority queue (pq). You must use a pq as entries into the pq are sorted by their heuristic value. The head of the pq will be the node with the smallest heuristic value.

The starting node needs to be added into the `unexplored list`, this is important as the node needs to be expanded to get its neighbours. To enter values into the pq the `.put(value, data)` function is used, where the value is the Manhattan distance from the node n (starting node) to the goal.

The `found` dictionary needs to be created and, as explained before, the index of the `start` node needs to map to the value `None`

The `while` loop will keep the search algorithm going until every node is explored. If the while loop condition returns `false` then all nodes have been explored without finding the goal node and no path can be found.

The variable `current` refers to the node we are currently exploring. The current node can be obtained by taking the head of the unexplored pq. The head is used as this is the node that's closest to the goal. The function `.get()` will return the head of the pq and remove it.
Using the `.get()` will return a tuple in the format of `(heuristic value, node)`, because of this we need to obtain just the nodes coordinate. We can index into the tuple by using the `[]` brackets and the index value of `1`

Now we have the current node there is no point expanding it if it's the goal node! Because of this we need to check if the current node is the goal node and if so stop the searching. We can break out of the `while` loop by using the `break` keyword. 

If the current node isn't the goal then we need to obtain its neighbours and, if they're not already found, add them to the found dictionary and unexplored pq. 
We can obtain a nodes neighbours by using the `neighbours` function we made earlier. We are also able to iterate over each neighbour using a `for` loop.

Next we need to check if the neighbour has been found before, if so there's no point in finding it again. 
If the node has not previously been discovered we need to add it to the found dictionary using the neighbour as the key and current as the value. This gives us the mapping of parent -> child that we need. 
We also need to add the node to the pq. Again we can do this using the `.put(value, data)` function, using the `calculate_manhattan_distance` function to calculate the heuristic value.

The `draw_grid` and `sys.stdout.write` are printing functions and aren't needed to perform the search, but they can help visualise how the search works and how nodes are found and expanded.

Finally, we return the `found` dictionary as this contains all the node mappings from the start node to the end.

In [8]:
def greedy_search():
    unexplored = Q.PriorityQueue()
    unexplored.put((calculate_manhattan_distance(start, end), start))
    found = {start: None}

    while not unexplored.empty():
        current = unexplored.get()
        current = current[1]

        if current == end:
            break

        for next_node in neighbors(current):
            if next_node not in found:
                found[next_node] = current
                unexplored.put((calculate_manhattan_distance(next_node, end), next_node))
                
        draw_grid(current, found)
        sys.stdout.write("\n")

    return found

### Running the search

Here we can define all the parameters needed and run the search. 
We need to set the maps `width`, `height`, `walls` (optional), `start` node and the `end` node.
Next, we can run the search making sure we store the result of the function.
The `reconstruct_path` function will calculate the shortest path from the node mappings so we need to run that and store the result.
Lastly, we can print the result to observe the path the algorithm found.

In [9]:
width = 30
height = 17
#walls = [(9, 16), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 14), (9, 13), (9, 3), (9, 2), (9, 1), (9, 15)]
walls = [(9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 14), (9, 13), (9, 3), (8, 3), (7, 3), (6, 3), (5, 3), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (24, 9), (24, 11), (24, 10), (25, 11), (25, 9), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 10)]
start = (0, 8)
end = (25, 10)
parents = greedy_search()
path = reconstruct_path(parents)
draw_grid((-1, -1), path)


 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 C  V  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  .  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 .  .  .  .  .  .  .  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 

 V  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 S  V  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  V  V  V  .  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 V  V  V  C  V  .  .  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 
 V  V  V  V  .  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 .  .  .  .  .  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 S  V  .  .  #  .  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  V  V  V  V  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 V  V  V  V  V  V  V  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 

 V  V  V  V  V  V  V  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 
 V  V  V  V  V  V  V  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 .  .  .  .  .  V  V  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  V  C  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

 .  .  .  .  #  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  #  V  C  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 S  V  .  .  #  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  V  V  V  V  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 V  V  V  V  V  V  V  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 
 V  V  V  V  V  V  V  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 .  .  .  .  V  V  V  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  V  V  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  V  V  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  V  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  #  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 S  V  .  .  #  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  V  V  V  V  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  .

 V  V  V  V  V  V  V  #  #  #  V  C  V  .  .  .  .  .  .  .  .  .  .  .  #  E  .  .  .  . 
 V  V  V  V  V  V  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  #  #  .  .  .  . 
 .  .  .  .  V  V  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  V  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  V  V  V  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  V  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  #  #  #  #  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  .  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  .  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  #  .  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  .  .  .  #  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 S  V  .  .  #  V  V  #  .  #  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 V  V  V  V  V  V  V  #  .  #  V  V  V  V  V  V  V  V  V  V  V  V  .  .  #  #  .  .  .  . 
 V  V  V  V  V  V  V  #  #  #  V  V  V  V  V  V  V  V  V  V  V  C  V  .  #  E  .  .  .  . 
 V  V  V  V  V  V  V  V  V  #  V  V  V  V  V  V  V  V  V  V  V  V  .  .  #  #  .  .  .  . 

 V  V  V  V  V  V  V  #  .  #  V  V  V  V  V  V  V  V  V  V  V  V  V  V  #  #  .  .  .  . 
 V  V  V  V  V  V  V  #  #  #  V  V  V  V  V  V  V  V  V  V  V  V  V  V  #  E  .  .  .  . 
 V  V  V  V  V  V  V  V  V  #  V  V  V  V  V  V  V  V  V  V  V  V  V  V  #  #  .  .  .  . 
 .  .  .  .  V  V  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  V  V  .  .  .  .  .  . 
 .  .  .  .  .  V  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  V  V  V  #  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  V  V  V  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  V  V  V  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .